In [30]:
import duckdb
import altair as alt
import polars as pl

con = duckdb.connect("riot_ci_stats.duckdb", read_only=True)

In [94]:
# con.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'main'").fetchall()

con.sql("SELECT table_name, column_name, data_type FROM information_schema.columns WHERE table_schema = 'main' ORDER BY table_name, ordinal_position").show()

┌──────────────┬────────────────────┬───────────┐
│  table_name  │    column_name     │ data_type │
│   varchar    │      varchar       │  varchar  │
├──────────────┼────────────────────┼───────────┤
│ jobs         │ uid                │ VARCHAR   │
│ jobs         │ commit_sha         │ VARCHAR   │
│ jobs         │ commit_message     │ VARCHAR   │
│ jobs         │ commit_author      │ VARCHAR   │
│ jobs         │ creation_time      │ TIMESTAMP │
│ jobs         │ start_time         │ TIMESTAMP │
│ jobs         │ total_tasks_count  │ INTEGER   │
│ jobs         │ failed_tasks_count │ INTEGER   │
│ jobs         │ passed_tasks_count │ INTEGER   │
│ jobs         │ runtime            │ DOUBLE    │
│  ·           │    ·               │   ·       │
│  ·           │    ·               │   ·       │
│  ·           │    ·               │   ·       │
│ worker_stats │ job_uid            │ VARCHAR   │
│ worker_stats │ name               │ VARCHAR   │
│ worker_stats │ tasks_count        │ INTEGER   │


## Metadata

In [6]:
ci_jobs_count = con.execute("SELECT COUNT(*) FROM jobs").fetchone()
ci_jobs_per_state = con.execute("SELECT state, COUNT(*) FROM jobs GROUP BY state").fetchall()

total_tasks_executed = con.execute("SELECT SUM(tasks_count) FROM worker_stats").fetchone()

print(f"Number of ci jobs that have been collected so far: {ci_jobs_count[0]}")
# print(f"{ci_jobs_per_state}")
print("number of jobs per state")
con.sql("SELECT state, COUNT(*) as count FROM jobs GROUP BY state order by count").show()
print(f"total executed build and test tasks: {total_tasks_executed[0]}")

Number of ci jobs that have been collected so far: 564
number of jobs per state
┌─────────┬───────┐
│  state  │ count │
│ varchar │ int64 │
├─────────┼───────┤
│ queued  │     4 │
│ running │     6 │
│ NULL    │    22 │
│ stopped │    39 │
│ errored │   215 │
│ passed  │   278 │
└─────────┴───────┘

total executed build and test tasks: 16709903


## CI average runtime

In [106]:
# Calculate weighted average over all CI runs with job URLs
query = """
SELECT
  IF(w.name LIKE 'ZIH-HPC-node%', 'ZIH-node', w.name) AS worker_name,
  j.url AS job_url,
  SUM(w.runtime_avg_s * w.tasks_count) / SUM(w.tasks_count) AS "avg_runtime"
FROM worker_stats w
JOIN jobs j ON w.job_uid = j.uid
WHERE w.tasks_count IS NOT NULL AND w.tasks_count != 0
GROUP BY worker_name, job_url
ORDER BY avg_runtime
"""

df_runtime = con.execute(query).fetch_df()

# Filter out specific Raspberry Pi worker nodes
df_runtime = df_runtime[~df_runtime['worker_name'].isin([
    'pi-68cf63c4',
    'pi-ef3229ad'
])]

# Facet the chart by job URL
chart = alt.Chart(df_runtime).mark_bar().encode(
  x=alt.X('worker_name:O', title='Worker Name'),
  y=alt.Y('avg_runtime:Q', title='Average Runtime'),
  tooltip=['worker_name', 'avg_runtime']
).facet(
  column=alt.Column('job_url:N', title='Average Runtime per worker')
)

chart.resolve_scale(x='independent')

alt.FacetChart(...)

In [8]:
# Calculate weighted average over all CI runs with job URLs
query = """
SELECT
  IF(w.name LIKE 'ZIH-HPC-node%', 'ZIH-node', w.name) AS worker_name,
  j.url AS job_url,
  SUM(w.tasks_count) "total_executed_tasks"
FROM worker_stats w
JOIN jobs j ON w.job_uid = j.uid
WHERE w.tasks_count IS NOT NULL AND w.tasks_count != 0
GROUP BY worker_name, job_url
"""

df_tasks = con.execute(query).fetch_df()

# Filter out specific Raspberry Pi worker nodes
df_tasks = df_tasks[~df_tasks['worker_name'].isin([
    'pi-68cf63c4',
    'pi-ef3229ad'
])]

# Chart total executed tasks per worker, faceted by job URL
chart_tasks = alt.Chart(df_tasks).mark_bar().encode(
    x=alt.X('worker_name:O', title='Worker Name'),
    y=alt.Y('total_executed_tasks:Q', title='Total Executed Tasks'),
    tooltip=['worker_name', 'total_executed_tasks']
).facet(
    column=alt.Column('job_url:N', title='Total executed tasks per worker')
)

chart_tasks

alt.FacetChart(...)

In [9]:
df_tasks.sort_values(by='total_executed_tasks')

,worker_name,job_url,total_executed_tasks
2,ZIH-node,https://ci.riot-os.org,16044.0
0,tatooine-staging,https://ci-staging.riot-os.org,29434.0
6,ZIH-node,https://ci-staging.riot-os.org,56746.0
3,tatooine,https://ci.riot-os.org,714007.0
10,mobi3,https://ci.riot-os.org,856468.0
9,breeze,https://ci.riot-os.org,1784755.0
4,mobi7,https://ci.riot-os.org,2144625.0
7,alien,https://ci.riot-os.org,4374259.0
5,mobi6,https://ci.riot-os.org,6729328.0


## Task statistics for each worker

In [93]:
query = """
SELECT
  IF(t.worker_name LIKE 'ZIH-HPC-node%', 'ZIH-node', t.worker_name) AS worker_name_normalized,
  quantile_cont(t.runtime_s, 0.05) as lower,
  quantile_cont(t.runtime_s, 0.25) as twenty_five,
  quantile_cont(t.runtime_s, 0.5) as avg,
  quantile_cont(t.runtime_s, 0.75) as seventy_five,
  quantile_cont(t.runtime_s, 0.95) as upper,
  toolchain,
  count(*) as count
FROM tasks_stats t
WHERE toolchain is not null
GROUP BY worker_name_normalized, toolchain
"""

tasks_runtime_distribution = con.sql(query).pl()

base = alt.Chart(tasks_runtime_distribution).encode(
    y="toolchain:N",
)

rules = base.mark_rule().encode(
    x=alt.X("lower").title("Runtime in seconds"),
    x2="upper",
)

bars = base.mark_bar(size=14).encode(
    x="twenty_five",
    x2="seventy_five",
    color=alt.Color("toolchain").legend(None),
)

ticks = base.mark_tick(color="white", size=14).encode(
    x="avg"
)

ticks_lower = base.mark_tick(size=14, color="black").encode(
    x="lower",
)

ticks_upper = base.mark_tick(size=14, color="black").encode(
    x="upper",
)


print(tasks_runtime_distribution.select(["worker_name_normalized", "toolchain", "count"]))

(rules + bars + ticks + ticks_lower + ticks_upper).facet(row=alt.Row("worker_name_normalized", title=""), title="distribution of task runtimes across worker nodes")

shape: (4, 3)
┌────────────────────────┬───────────┬───────┐
│ worker_name_normalized ┆ toolchain ┆ count │
│ ---                    ┆ ---       ┆ ---   │
│ str                    ┆ str       ┆ i64   │
╞════════════════════════╪═══════════╪═══════╡
│ tatooine-staging       ┆ llvm      ┆ 6571  │
│ tatooine-staging       ┆ gnu       ┆ 8355  │
│ ZIH-node               ┆ llvm      ┆ 10967 │
│ ZIH-node               ┆ gnu       ┆ 14242 │
└────────────────────────┴───────────┴───────┘


alt.FacetChart(...)

In [125]:
query = """
SELECT
  IF(t.worker_name LIKE 'ZIH-HPC-node%', 'ZIH-node', t.worker_name) AS worker_name_normalized,
  quantile_cont(t.runtime_s, 0.01) as lower,
  quantile_cont(t.runtime_s, 0.25) as twenty_five,
  quantile_cont(t.runtime_s, 0.5) as avg,
  quantile_cont(t.runtime_s, 0.75) as seventy_five,
  quantile_cont(t.runtime_s, 0.99) as upper,
  application
FROM tasks_stats t
GROUP BY worker_name_normalized, application
"""

tasks_df = con.sql(query).pl()

tasks_df = tasks_df.sort("avg", descending=True)

[ longest_tasks_zih ] = tasks_df \
            .filter(pl.col("worker_name_normalized").eq(pl.lit("ZIH-node")))\
            .head(10),

base = alt.Chart(longest_tasks_zih, width=600,title="avg longest tasks runtime distribution [ZIH]" ).encode(
    y=alt.Y("application:N",sort=alt.EncodingSortField(field="avg")),
)

rules = base.mark_rule().encode(
    x=alt.X("lower").title("Runtime in seconds"),
    x2="upper",
)

ticks_lower = base.mark_tick(size=14, color="black").encode(
    x="lower",
)

ticks_upper = base.mark_tick(size=14, color="black").encode(
    x="upper",
)

bars = base.mark_bar(size=14).encode(
    x="twenty_five",
    x2="seventy_five",
)

ticks = base.mark_tick(color="white", size=14).encode(
    x="avg"
)



tasks_chart_zih = (rules + bars + ticks + ticks_lower + ticks_upper)


longest_tasks_tatooine = tasks_df \
            .filter(pl.col("worker_name_normalized").eq(pl.lit("tatooine-staging"))) \
            .head(10)

base = alt.Chart(longest_tasks_tatooine, title="avg longest tasks runtime distribution [tatooine]").encode(
    y=alt.Y("application:N",sort=alt.EncodingSortField(field="avg")),
)

rules = base.mark_rule().encode(
    x=alt.X("lower").title("Runtime in seconds"),
    x2="upper",
)

ticks_lower = base.mark_tick(size=14, color="black").encode(
    x="lower",
)

ticks_upper = base.mark_tick(size=14, color="black").encode(
    x="upper",
)

bars = base.mark_bar(size=14).encode(
    x="twenty_five",
    x2="seventy_five",
)

ticks = base.mark_tick(color="white", size=14).encode(
    x="avg"
)



tasks_chart_tatooine = (rules + bars + ticks + ticks_lower + ticks_upper)


(tasks_chart_tatooine & tasks_chart_zih).resolve_scale(x='shared')

alt.VConcatChart(...)

In [111]:
tasks_df = tasks_df.sort("avg_runtime", descending=True)

[ longest_tasks_zih ] = tasks_df \
            .filter(pl.col("worker_name_normalized").eq(pl.lit("ZIH-node")))\
            .head(10),


tasks_chart_zih = alt.Chart(longest_tasks_zih, width=600, title="longest tasks [ZIH-worker]").mark_bar().encode(
    x=alt.X("max_runtime:Q", title="average runtime in s"),
    y=alt.Y("application:N", sort="-x"),
)

longest_tasks_tatooine = tasks_df \
            .filter(pl.col("worker_name_normalized").eq(pl.lit("tatooine-staging"))) \
            .head(10)


tasks_chart_tatooine = alt.Chart(longest_tasks_tatooine, width=600, title="longest tasks [tatooine]").mark_bar().encode(
    x=alt.X("max_runtime:Q", title="average runtime in s"),
    y=alt.Y("application:N", sort="-x"),
)

(tasks_chart_tatooine & tasks_chart_zih).resolve_scale(x='shared')

alt.VConcatChart(...)

## Cleanup

In [12]:
con.close()